In [1]:
import pandas as pd
import talib as ta
import os
from ftx_client import FtxClient
import supertrend as spt
import backtesting as bt
from config import API_KEY, API_SECRET
import itertools
from typing import Tuple
from tqdm import tqdm

In [2]:
BACKTEST_FOLDER = "./backtest"
if not os.path.exists(BACKTEST_FOLDER):
    os.makedirs(BACKTEST_FOLDER)

ANALYSIS_FILEPATH = os.path.join(BACKTEST_FOLDER, "MarketAnalysis.csv")
OPTIMIZEDML_FILEPATH = os.path.join(BACKTEST_FOLDER, "OptimizedML_40days_median.csv")

In [3]:
ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

In [4]:
markets = []
for future in ftx.list_futures():
    if future["type"] == "perpetual":
        if future["volumeUsd24h"] > 1e7:
            markets.append(future["name"])

In [5]:
optimized_params = pd.DataFrame(columns=["Name", "Multiplier", "Lookback"])

for market in tqdm(markets):
    df = ftx.get_historical_market_data(market, interval="4h", start_time="40 days ago")

    if len(df) < 240:
        continue
    
    optimized_result = bt.optimize_m_l(df, optimize_to="MedPosNegRetRatio")
    optimized_result["Name"] = market
    optimized_params = optimized_params.append(optimized_result, ignore_index=True)

  0%|          | 0/36 [00:00<?, ?it/s]/home/dineshpinto/miniconda3/envs/swingaroo/lib/python3.8/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/dineshpinto/miniconda3/envs/swingaroo/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/dineshpinto/miniconda3/envs/swingaroo/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dineshpinto/miniconda3/envs/swingaroo/lib/python3.8/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/dineshpinto/miniconda3/envs/swingaroo/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/dineshpinto/miniconda3/envs/swingaroo/lib/pyth

In [6]:
optimized_params

,Name,Multiplier,Lookback
0,AAVE-PERP,2,9
1,ADA-PERP,3,9
2,ATOM-PERP,3,9
3,AUDIO-PERP,2,10
4,AXS-PERP,3,9
5,BCH-PERP,4,9
6,BNB-PERP,2,9
7,BTC-PERP,2,9
8,CHZ-PERP,3,9
9,COMP-PERP,2,9


In [7]:
optimized_params.to_csv(OPTIMIZEDML_FILEPATH, index=False)